In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter


In [2]:
df=sns.load_dataset("iris")

In [3]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [4]:
(df["species"]).unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [5]:
# Map species to numeric
class_map = {"setosa": 0, "versicolor": 1, "virginica": 2}
class_unmap = {0:"setosa",1: "versicolor", 2:"virginica"}
df["species"] = df["species"].replace(class_map)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9664\918621057.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["species"] = df["species"].replace(class_map)


# split

In [6]:
def one_hot_encode(labels):
    classes = sorted(set(labels))
    class_to_idx = {c: i for i, c in enumerate(classes)}
    Y = np.zeros((len(labels), len(classes)))
    for i, label in enumerate(labels):
        Y[i, class_to_idx[label]] = 1
    return Y, class_to_idx

In [7]:
def split(X, y, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1."

    np.random.seed(42)
    indices = np.arange(len(X))
    np.random.shuffle(indices)

    # Ensure NumPy arrays
    X = np.array(X)
    y = np.array(y)

    X = X[indices]
    y = y[indices]

    n_train = int(train_ratio * len(X))
    n_val = int(val_ratio * len(X))

    X_train, y_train = X[:n_train], y[:n_train]
    X_val, y_val = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
    X_test, y_test = X[n_train+n_val:], y[n_train+n_val:]

    return X_train, y_train, X_val, y_val, X_test, y_test


In [8]:
X=df.drop(columns="species")
y=df["species"]

In [9]:
X=X.values
y=y.values

In [20]:

def train(X,y):
    # Add bias term
    # X=X.values
    # X_b = np.c_[np.ones((X.shape[0], 1)), X]  
    X_b=X

    # Normal Equation
    theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y

    # Predictions
    y_pred = X_b @ theta

    # Error Metrics
    MAE = np.mean(np.abs(y - y_pred))
    MSE = np.mean((y - y_pred)**2)
    R2 = 1 - np.sum((y - y_pred)**2) / np.sum((y - np.mean(y))**2)

    print(f"MAE:{MAE:.4f}")
    print(f"MSE:{MSE:.4f}")
    print(f"R²:{R2:.4f}")

    return theta


In [11]:
X_train, y_train, X_val, y_val, X_test, y_test = split(X, y)

In [21]:
w_train=train(X_train,y_train)
print("------------------------------")
w_val=train(X_val,y_val)
print("------------------------------")
w_test=train(X_test,y_test)

MAE:0.1562
MSE:0.0468
R²:0.9312
------------------------------
MAE:0.1644
MSE:0.0380
R²:0.9292
------------------------------
MAE:0.1200
MSE:0.0226
R²:0.9639
